In [1]:
import pandas as pd

# Cuota parte

In [2]:
def cuota_parte(cuota,capacidad,SA,PE,SIN,Rea):
    """Función que calcula el contrato de reaguro cuota parte
    
    Inputs:
    Cuota: float
    
    Es el porcentaje de la cuota parte
    
    Capacidad: float
    
    Es la capacidad del contrato
    
    SA: float
    
    Es la suma asegurada
    
    PE: float
    
    Es el valor de la prima emitida
    
    SIN: float
    
     Es el valor del siniestro
     
     Rea:list
     
     Lista con los porcentajes de las reaseguradoras
    
    """
    minimo=min(cuota*SA,cuota*capacidad)
    if minimo==cuota*SA:
        Sa=[minimo]
        for ase in Rea:
            Sa+=[(SA-Sa[0])*ase]
        porcentajes=[x/SA for x in Sa]
        primas=[p*PE for p in porcentajes]
        siniestros=[p*SIN for p in porcentajes]
        columnas=["Retención"]+["CP Reaseguradora "+str(r+1) for r  in range(len(Rea))]

    else:
        facultativo=SA-capacidad
        Sa=[capacidad*cuota]
        for ase in Rea:
                Sa+=[capacidad*(1-cuota)*ase]
        Sa+=[facultativo]
        porcentajes=[x/SA for x in Sa]
        primas=[p*PE for p in porcentajes]
        siniestros=[p*SIN for p in porcentajes]
        columnas=["Retención"]+["CP Reaseguradora "+str(r+1) for r  in range(len(Rea))]+['Facultativo']

    df= pd.DataFrame([Sa,primas,siniestros],columns=columnas).T
    df.columns=['Suma Asegurada','Prima','Siniestro']
    return df

In [3]:
#Ejemplo 1
cuota_parte(0.4,2_000_000,600_000,25_000,120_000,[0.3,0.7])

,Suma Asegurada,Prima,Siniestro
Retención,240000.0,10000.0,48000.0
CP Reaseguradora 1,108000.0,4500.0,21600.0
CP Reaseguradora 2,252000.0,10500.0,50400.0


In [4]:
#Ejempolo 2
cuota_parte(0.4,2_000_000,600_000,25_000,120_000,[1,0])

,Suma Asegurada,Prima,Siniestro
Retención,240000.0,10000.0,48000.0
CP Reaseguradora 1,360000.0,15000.0,72000.0
CP Reaseguradora 2,0.0,0.0,0.0


In [5]:
#Ejempolo 3
cuota_parte(0.6,2_000_000,3_500_000,25_000,250_000,[0.4,0.6])

,Suma Asegurada,Prima,Siniestro
Retención,1200000.0,8571.428571,85714.285714
CP Reaseguradora 1,320000.0,2285.714286,22857.142857
CP Reaseguradora 2,480000.0,3428.571429,34285.714286
Facultativo,1500000.0,10714.285714,107142.857143


In [6]:
#Ejemplo 4
cuota_parte(0.4,2_000_000,2_500_000,70_000,500_000,[0.3,0.7])

,Suma Asegurada,Prima,Siniestro
Retención,800000.0,22400.0,160000.0
CP Reaseguradora 1,360000.0,10080.0,72000.0
CP Reaseguradora 2,840000.0,23520.0,168000.0
Facultativo,500000.0,14000.0,100000.0


In [7]:
#Ejemplo 5
cuota_parte(0.3,3_000_000,4_500_000,100_000,800_000,[0.4,0.5,0.1])

,Suma Asegurada,Prima,Siniestro
Retención,900000.0,20000.000000,160000.000000
CP Reaseguradora 1,840000.0,18666.666667,149333.333333
CP Reaseguradora 2,1050000.0,23333.333333,186666.666667
CP Reaseguradora 3,210000.0,4666.666667,37333.333333
Facultativo,1500000.0,33333.333333,266666.666667


# Excedentes

In [8]:
def excedentes(limite, lineas,SA,PE,Sin,REA):
    """
    Función que calcula los contratos de excedentes 
    
    inputs:
    
    limite: float
    
    Es el limite de rentención por líneas
    
    lineas: float
    
    Es el limite de lineas que tiene el contrato
    
    SA: float
    
    Es el valor de la suma asegurada
    
    PE:float
    
    Es el valor de la prima emitida
    
    Sin: foat
    
    Es el valor del siniestro
    
    REA:list
    
    Lista con los porcentajes de las reaseguradoras
    
    Outpout:
    DataFrame
    """
    capacidad=limite*lineas
    if sum(REA)>1:
        REA=[r/100 for r in REA]
    if capacidad>SA:
        exceden=[limite]+[(SA-limite)*re for re in REA]
        columnas=["Retención"]+['Excedente Reaseguradora '+str(i+1) for i in range(len(REA))]
    else:
        exceden=[limite]+[(capacidad)*re for re in REA]
        facultativo=SA-sum(exceden)
        exceden=exceden+[facultativo]
        columnas=["Retención"]+['Excedente Reaseguradora '+str(i+1) for i in range(len(REA))]+['Facultativo']
    porcentajes=[exc/SA for exc in exceden]
    primas=[PE*por for por in porcentajes]
    siniestros=[Sin*por for por in porcentajes]

    df= pd.DataFrame([exceden,primas,siniestros],columns=columnas).T
    df.columns=['Suma Asegurada','Prima','Siniestro']
    
    return df

In [9]:
#Ejemplo 1
excedentes(500_000,10,600_000,25_000,300_000,[1])

,Suma Asegurada,Prima,Siniestro
Retención,500000.0,20833.333333,250000.0
Excedente Reaseguradora 1,100000.0,4166.666667,50000.0


In [10]:
#Ejemplo 2
excedentes(500_000,10, 3_000_000,80_000, 300_000,[35,65])

,Suma Asegurada,Prima,Siniestro
Retención,500000.0,13333.333333,50000.0
Excedente Reaseguradora 1,875000.0,23333.333333,87500.0
Excedente Reaseguradora 2,1625000.0,43333.333333,162500.0


In [11]:
#Ejemplo 3
excedentes(500_000,10,7_000_000,160_000,300_000 ,[20,30,50])

,Suma Asegurada,Prima,Siniestro
Retención,500000.0,11428.571429,21428.571429
Excedente Reaseguradora 1,1000000.0,22857.142857,42857.142857
Excedente Reaseguradora 2,1500000.0,34285.714286,64285.714286
Excedente Reaseguradora 3,2500000.0,57142.857143,107142.857143
Facultativo,1500000.0,34285.714286,64285.714286


In [12]:
#Ejemplo 4
excedentes(600_000,11,10_000_000,280_000,900_000,[25,35,30,10])

,Suma Asegurada,Prima,Siniestro
Retención,600000.0,16800.0,54000.0
Excedente Reaseguradora 1,1650000.0,46200.0,148500.0
Excedente Reaseguradora 2,2310000.0,64680.0,207900.0
Excedente Reaseguradora 3,1980000.0,55440.0,178200.0
Excedente Reaseguradora 4,660000.0,18480.0,59400.0
Facultativo,2800000.0,78400.0,252000.0
